In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import csv
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
import os
print(os.getcwd())

d:\IML ASSIGNMENT 2\CODE


In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]

In [6]:
object_cols = X.select_dtypes(include=['object']).columns

In [7]:
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_12296\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_12296\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_12296\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [8]:
X.shape

(178661, 271)

In [9]:
num_cols = X.select_dtypes(include=['int64','float64']).columns

In [10]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

scaler = RobustScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_12296\334397748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = scaler.fit_transform(X[num_cols])


In [11]:
X.shape

(178661, 271)

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=75)
X = pca.fit_transform(X)

In [13]:
# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=3)
# X = poly.fit_transform(X)

In [14]:
X.shape

(178661, 75)

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [16]:
df_test = pd.read_csv('test.csv')

In [17]:
df_test = df_test.drop(['row ID'], axis=1)

In [18]:
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))


In [19]:
df_test.shape

(77789, 271)

In [20]:
df_test[num_cols] = scaler.transform(df_test[num_cols])

In [21]:
df_test = pca.transform(df_test)

In [22]:
# df_test = poly.transform(df_test)

In [23]:
df_test.shape

(77789, 75)

In [24]:
#convert all int64 to float32
X = X.astype('float32')
df_test = df_test.astype('float32')

In [26]:
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import RMSprop,Adam,Adamax
from keras.callbacks import EarlyStopping

In [27]:
# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(hidden_layer_sizes=(128,64,32,16,8), max_iter=200, solver='adam', verbose=True, random_state=42,
#                    batch_size=1300,early_stopping=True)
# mlp.fit(X, y)
# y_pred = mlp.predict(df_test)

In [28]:
n_features = X.shape[1]

# Create a Sequential model
model = Sequential()

# Add the first hidden layer with 128 neurons and specify the input shape
model.add(Dense(128, input_dim=n_features, activation='sigmoid', kernel_regularizer=l2(0.01)))


# Add the second hidden layer with 100 neurons
model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01)))


# Add the third hidden layer with 64 neurons
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))


# Add the output layer
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Print the model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               9728      
                                                                 
 dense_1 (Dense)             (None, 100)               12900     
                                                                 
 dense_2 (Dense)             (None, 64)                6464      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 29157 (113.89 KB)
Trainable params: 29157 (113.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
# get the number of input features in X and assign to n_features
# n_features = X.shape[1]
# Create a Sequential model
# model = Sequential()

# Add the first hidden layer with 10 neurons and specify the input shape
# model.add(Dense(128, input_dim=n_features, activation='sigmoid',kernel_regularizer=l2(0.01)))

# Add the second hidden layer with 5 neurons
# model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01)))

# Add the third hidden layer with 3 neurons
# model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.01)))


# Compile the model
# model.compile(optimizer=Adam(), loss='mean_squared_error') #change optimizer

# Print the model summary
# print(model.summary())


In [30]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, epochs=200, batch_size=512, callbacks=[early_stopping])
y_pred = model.predict(df_test)

Epoch 1/200

349/349 [==============================] - 2s 3ms/step - loss: nan
Epoch 2/200
349/349 [==============================] - 1s 3ms/step - loss: nan
Epoch 3/200
2431/2431 [==============================] - 2s 906us/step


In [31]:
test_pred = model.predict(X_test)
print("root_mean_squared_error: ",mean_squared_error(y_test,test_pred,squared=False))

1675/1675 [==============================] - 2s 1ms/step
root_mean_squared_error:  26719809.079837583


In [33]:
filepath = 'neural_2dec_1.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c=1
    writer.writerow(['row ID', 'price_doc'])
    for i in y_pred:
        writer.writerow([c ,i[0]])
        c=c+1